In [78]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
random.seed(3)
import json

torch.manual_seed(1)

import pandas as pd
import numpy as np
import re
import itertools
import ast
from tqdm import tqdm_notebook
from sklearn.metrics import classification_report, accuracy_score, f1_score, balanced_accuracy_score
from sklearn.model_selection import train_test_split
import itertools

from gensim.models import Word2Vec
import matplotlib.pyplot as plt
%matplotlib inline

from snorkel.labeling import labeling_function
from snorkel.labeling import LabelModel, PandasLFApplier

#### Data

In [79]:
with open('/Users/prakharagarwal/Desktop/Snorkel/Data/recipes_raw/recipes_raw_nosource_ar.json') as json_file:
    data = json.load(json_file)

In [80]:
data_set = []
for key in (data.keys()):
    try:
        recipe_list = data[key]["ingredients"][:-1]
        data_set.extend([i[:-14].replace(",","").replace(":","") for i in recipe_list]) #removing the word ADVERTISEMENT
    except (KeyError):
        continue

In [81]:
data_set[0:10]

['4 skinless boneless chicken breast halves',
 '2 tablespoons butter',
 '2 (10.75 ounce) cans condensed cream of chicken soup',
 '1 onion finely diced',
 '2 (10 ounce) packages refrigerated biscuit dough torn into pieces',
 '2 (10.75 ounce) cans condensed cream of mushroom soup',
 '1 (1 ounce) package dry onion soup mix',
 '1 1/4 cups water',
 '5 1/2 pounds pot roast',
 '1/2 cup packed brown sugar']

In [82]:
df_data_set = pd.DataFrame({'text':data_set, 'label':[-1]*len(data_set)})
df_data_set

,text,label
0,4 skinless boneless chicken breast halves,-1
1,2 tablespoons butter,-1
2,2 (10.75 ounce) cans condensed cream of chicke...,-1
3,1 onion finely diced,-1
4,2 (10 ounce) packages refrigerated biscuit dou...,-1
...,...,...
361319,1/2 cup chopped almonds,-1
361320,1/2 cup flax seed meal,-1
361321,1/2 cup hemp seed hearts,-1
361322,1/4 cup hemp seeds,-1


In [83]:
df_data_set_sample = [_text for _text in df_data_set.text if _text.count(" ") == 2]

In [84]:
df_data_set_sample = pd.DataFrame({'text':df_data_set_sample, 'label':[-1]*len(df_data_set_sample)}) 
df_data_set_sample.head()

,text,label
0,2 tablespoons butter,-1
1,1/2 cup ketchup,-1
2,3/4 cup milk,-1
3,1/2 teaspoon salt,-1
4,1/4 teaspoon salt,-1


In [86]:
train_df, test_df = train_test_split(df_data_set_sample, test_size=0.3, random_state = 42)

####  Labeling Functions

In [87]:
ABSTAIN = -1
INGREDIENT = 1
NotINGREDIENT = 0

In [88]:
@labeling_function()
def lf_step_split(x):
    """identify ingredient in a step"""
    val = x.text.split(" ", 3)
    return INGREDIENT if (val[0].isnumeric() and len(val) == 3) else ABSTAIN

@labeling_function()
def lf_step_split_fraction(x):
    """identify ingredient in a step"""
    val = x.text.split(" ", 3)
    return INGREDIENT if (bool(re.match(r"\d+\/\d+", val[0])) and len(val) == 3) else ABSTAIN

In [90]:
# Define the set of labeling functions (LFs)
lfs = [lf_step_split, lf_step_split_fraction]
# Apply the LFs to the unlabeled training data
applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=train_df)
L_dev = applier.apply(df=test_df)

100%|██████████| 17300/17300 [00:01<00:00, 12127.66it/s]


In [91]:
coverage_split, coverage_split_fraction = (L_train != ABSTAIN).mean(axis=0)
print(f"split coverage: {coverage_split * 100:.1f}%")
print(f"split_fraction coverage: {coverage_split_fraction * 100:.1f}%")

split coverage: 62.7%
split_fraction coverage: 33.1%


In [92]:
from snorkel.labeling import LFAnalysis

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
lf_step_split,0,[1],0.627193,0.0,0.0
lf_step_split_fraction,1,[1],0.331112,0.0,0.0


In [94]:
# LFAnalysis(L=L_dev, lfs=lfs).lf_summary(Y=Y_dev)

In [27]:
# def lf_step_split_df(x):
#     """identify ingredient in a step"""
#     val = x.split(" ", 3)
#     if len(val)==3:
#         return [0,0,1]
#     else:
#         return -1
# label = list(df_data_set["text"].apply(lf_step_split_df))

In [34]:
L_train

array([[ 1],
       [-1],
       [-1],
       ...,
       [-1],
       [-1],
       [ 1]])

In [17]:
label

[-1,
 [0, 0, 1],
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 [0, 0, 1],
 -1,
 [0, 0, 1],
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 [0, 0, 1],
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 [0, 0, 1],
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 [0, 0, 1],
 [0, 0, 1],
 -1,
 [0, 0, 1],
 -1,
 [0, 0, 1],
 -1,
 [0, 0, 1],
 -1,
 -1,
 -1,
 [0, 0, 1],
 [0, 0, 1],
 -1,
 -1,
 [0, 0, 1],
 [0, 0, 1],
 -1,
 -1,
 -1,
 -1,
 [0, 0, 1],
 [0, 0, 1],
 -1,
 -1,
 -1,
 [0, 0, 1],
 -1,
 -1,
 [0, 0, 1],
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 [0, 0, 1],
 -1,
 [0, 0, 1],
 -1,
 [0, 0, 1],
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 [0, 0, 1],
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 [0, 0, 1],
 -1,
 -1,
 -1,
 -1,
 -1,
 [0, 0, 1],
 -1,
 -1,
 -1,
 -1,
 [0, 0, 1],
 -1,
 -1,
 -1,
 -1,
 [0, 0, 1],
 -1,
 -1,
 -1,
 -1,
 [0, 0, 1],
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 [0, 0, 1],
 -1,
 -1,
 -1,
 [0, 0, 1],
 -1,
 -1,
 -1,
 [

In [18]:
_df_data_set = df_data_set.copy()

In [19]:
_df_data_set["label"] = label
_df_data_set

,text,label
0,4 skinless boneless chicken breast halves,-1
1,2 tablespoons butter,"[0, 0, 1]"
2,2 (10.75 ounce) cans condensed cream of chicke...,-1
3,1 onion finely diced,-1
4,2 (10 ounce) packages refrigerated biscuit dou...,-1
...,...,...
361319,1/2 cup chopped almonds,-1
361320,1/2 cup flax seed meal,-1
361321,1/2 cup hemp seed hearts,-1
361322,1/4 cup hemp seeds,-1


In [20]:
df_test = _df_data_set[_df_data_set['label'] != -1]
train_df, val_df = train_test_split(df_test, test_size=0.10, random_state = 42)

In [21]:
complete_set = []
for index, row in tqdm_notebook(df_test.iterrows()):
    complete_set.append((row["text"].split(), row["label"]))

In [22]:
train_set = []
for index, row in tqdm_notebook(train_df.iterrows()):
    train_set.append((row["text"].split(), row["label"]))

In [23]:
val_set = []
for index, row in tqdm_notebook(val_df.iterrows()):
    val_set.append((row["text"].split(), row["label"]))

#### LSTM

In [ ]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

In [ ]:
word_to_ix = {}
for sent, tags in complete_set:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
# DEBUG :
# print(word_to_ix)
tag_to_ix = {"0": 0, 
             "1": 1}

In [ ]:
word_to_ix

In [ ]:
EMBEDDING_DIM = 300
HIDDEN_DIM = 128
NUM_EPOCH = 10

In [ ]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [ ]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

train_micro_f1_model1 = []
train_macro_f1_model1 = []
train_weighted_f1_model1 = []
train_balanced_accuracy_model1 = []
val_micro_f1_model1 = []
val_macro_f1_model1 = []
val_weighted_f1_model1 = []
val_balanced_accuracy_model1 = []


for epoch in tqdm_notebook(range(NUM_EPOCH)):
    print (epoch)
    for sentence, tags in tqdm_notebook(train_set):
        model.zero_grad()
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        tag_scores = model(sentence_in)
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        
    # Train score
    y_true = []
    y_pred = []
    for sentence, tags in train_set:
        y_true.extend(prepare_sequence(tags, tag_to_ix).tolist())
        tag_scores = model(prepare_sequence(sentence, word_to_ix))
        for score in tag_scores:
            val = max(range(len(score)), key = lambda x: score[x])
            y_pred.append(val)

    train_micro_f1_model1.append(f1_score(y_true, y_pred, average='micro'))
    train_macro_f1_model1.append(f1_score(y_true, y_pred, average='macro'))
    train_weighted_f1_model1.append(f1_score(y_true, y_pred, average='weighted'))
    train_balanced_accuracy_model1.append(balanced_accuracy_score(y_true, y_pred))
    if (epoch == NUM_EPOCH-1):
        print ("Classification Report Training")
        print(classification_report(y_true, y_pred))
    
    # Val score
    y_true = []
    y_pred = []
    for sentence, tags in val_set:
        y_true.extend(prepare_sequence(tags, tag_to_ix).tolist())
        tag_scores = model(prepare_sequence(sentence, word_to_ix))
        for score in tag_scores:
            val = max(range(len(score)), key = lambda x: score[x])
            y_pred.append(val)
    val_micro_f1_model1.append(f1_score(y_true, y_pred, average='micro'))r
    val_macro_f1_model1.append(f1_score(y_true, y_pred, average='macro'))
    val_weighted_f1_model1.append(f1_score(y_true, y_pred, average='weighted'))
    val_balanced_accuracy_model1.append(balanced_accuracy_score(y_true, y_pred))
    if (epoch == NUM_EPOCH-1):
        print ("Classification Report Validation")
        print(classification_report(y_true, y_pred))

#### LSTM pre trained embeddings (False)

In [ ]:
_temp = Word2Vec.load("../Embedding/model/w2v_model.gensim")

In [ ]:
temp_array = []
count = 0
for word in tqdm_notebook(word_to_ix):
    if word in _temp.wv.vocab.keys():
        temp_array.append(list(_temp[word]))
    else :
        count += 1
        temp_array.append(np.random.randn(1,300)[0])
count

In [ ]:
pretrained_weights = torch.FloatTensor(np.array(temp_array))

In [ ]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

#         self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.word_embeddings = nn.Embedding.from_pretrained(pretrained_weights)
        self.word_embeddings.weight.require_grad = False
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [ ]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)


train_micro_f1_model2 = []
train_macro_f1_model2 = []
train_weighted_f1_model2 = []
train_balanced_accuracy_model2 = []
val_micro_f1_model2 = []
val_macro_f1_model2 = []
val_weighted_f1_model2 = []
val_balanced_accuracy_model2 = []


for epoch in tqdm_notebook(range(NUM_EPOCH)):
    print (epoch)
    for sentence, tags in tqdm_notebook(train_set):
        model.zero_grad()
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        tag_scores = model(sentence_in)
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        
    # Train score
    y_true = []
    y_pred = []
    for sentence, tags in train_set:
        y_true.extend(prepare_sequence(tags, tag_to_ix).tolist())
        tag_scores = model(prepare_sequence(sentence, word_to_ix))
        for score in tag_scores:
            val = max(range(len(score)), key = lambda x: score[x])
            y_pred.append(val)

    train_micro_f1_model2.append(f1_score(y_true, y_pred, average='micro'))
    train_macro_f1_model2.append(f1_score(y_true, y_pred, average='macro'))
    train_weighted_f1_model2.append(f1_score(y_true, y_pred, average='weighted'))
    train_balanced_accuracy_model2.append(balanced_accuracy_score(y_true, y_pred))
    if (epoch == NUM_EPOCH-1):
        print ("Classification Report Training")
        print(classification_report(y_true, y_pred))
    
    # Val score
    y_true = []
    y_pred = []
    for sentence, tags in val_set:
        y_true.extend(prepare_sequence(tags, tag_to_ix).tolist())
        tag_scores = model(prepare_sequence(sentence, word_to_ix))
        for score in tag_scores:
            val = max(range(len(score)), key = lambda x: score[x])
            y_pred.append(val)
    val_micro_f1_model2.append(f1_score(y_true, y_pred, average='micro'))
    val_macro_f1_model2.append(f1_score(y_true, y_pred, average='macro'))
    val_weighted_f1_model2.append(f1_score(y_true, y_pred, average='weighted'))
    val_balanced_accuracy_model2.append(balanced_accuracy_score(y_true, y_pred))
    if (epoch == NUM_EPOCH-1):
        print ("Classification Report Validation")
        print(classification_report(y_true, y_pred))

#### LSTM pre trained embeddings (True)

In [ ]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

#         self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.word_embeddings = nn.Embedding.from_pretrained(pretrained_weights)
        self.word_embeddings.weight.require_grad = True
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [ ]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)


train_micro_f1_model3 = []
train_macro_f1_model3 = []
train_weighted_f1_model3 = []
train_balanced_accuracy_model3 = []
val_micro_f1_model3 = []
val_macro_f1_model3 = []
val_weighted_f1_model3 = []
val_balanced_accuracy_model3 = []


for epoch in tqdm_notebook(range(NUM_EPOCH)):
    print (epoch)
    for sentence, tags in tqdm_notebook(train_set):
        model.zero_grad()
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        tag_scores = model(sentence_in)
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        
    # Train score
    y_true = []
    y_pred = []
    for sentence, tags in train_set:
        y_true.extend(prepare_sequence(tags, tag_to_ix).tolist())
        tag_scores = model(prepare_sequence(sentence, word_to_ix))
        for score in tag_scores:
            val = max(range(len(score)), key = lambda x: score[x])
            y_pred.append(val)

    train_micro_f1_model2.append(f1_score(y_true, y_pred, average='micro'))
    train_macro_f1_model2.append(f1_score(y_true, y_pred, average='macro'))
    train_weighted_f1_model2.append(f1_score(y_true, y_pred, average='weighted'))
    train_balanced_accuracy_model2.append(balanced_accuracy_score(y_true, y_pred))
    if (epoch == NUM_EPOCH-1):
        print ("Classification Report Training")
        print(classification_report(y_true, y_pred))
    
    # Val score
    y_true = []
    y_pred = []
    for sentence, tags in val_set:
        y_true.extend(prepare_sequence(tags, tag_to_ix).tolist())
        tag_scores = model(prepare_sequence(sentence, word_to_ix))
        for score in tag_scores:
            val = max(range(len(score)), key = lambda x: score[x])
            y_pred.append(val)
    val_micro_f1_model3.append(f1_score(y_true, y_pred, average='micro'))
    val_macro_f1_model3.append(f1_score(y_true, y_pred, average='macro'))
    val_weighted_f1_model3.append(f1_score(y_true, y_pred, average='weighted'))
    val_balanced_accuracy_model3.append(balanced_accuracy_score(y_true, y_pred))
    if (epoch == NUM_EPOCH-1):
        print ("Classification Report Validation")
        print(classification_report(y_true, y_pred))

#### Bi-LSTM

In [ ]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

In [ ]:
class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [ ]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 128
HIDDEN_DIM = 128
NUM_EPOCH = 10

In [ ]:
word_to_ix = {}
for sentence, tags in complete_set:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

tag_to_ix = {"0": 0, 
     "1": 1, 
    START_TAG: 2, 
 STOP_TAG: 3}

In [ ]:
model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

In [ ]:
train_micro_f1_model4 = []
train_macro_f1_model4 = []
train_weighted_f1_model4 = []
train_balanced_accuracy_model4 = []
val_micro_f1_model4 = []
val_macro_f1_model4 = []
val_weighted_f1_model4 = []
val_balanced_accuracy_model4 = []


for epoch in tqdm_notebook(range(NUM_EPOCH)):
    print (epoch)
    for sentence, tags in tqdm_notebook(train_set):
        model.zero_grad()
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)
        loss = model.neg_log_likelihood(sentence_in, targets)
        loss.backward()
        optimizer.step()
        
    # Train score
    y_true = []
    y_pred = []    
    for sentence, tags in train_set:
        _targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long).tolist()
        y_true.extend(_targets)
        tag_scores = model(prepare_sequence(sentence, word_to_ix))[1]
        y_pred.extend(tag_scores)

    train_micro_f1_model4.append(f1_score(y_true, y_pred, average='micro'))
    train_macro_f1_model4.append(f1_score(y_true, y_pred, average='macro'))
    train_weighted_f1_model4.append(f1_score(y_true, y_pred, average='weighted'))
    train_balanced_accuracy_model4.append(balanced_accuracy_score(y_true, y_pred))
    if (epoch == NUM_EPOCH-1):
        print ("Classification Report Training")
        print(classification_report(y_true, y_pred))
    
    # Val score
    y_true = []
    y_pred = []
    for sentence, tags in val_set:
        _targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long).tolist()
        y_true.extend(_targets)
        tag_scores = model(prepare_sequence(sentence, word_to_ix))[1]
        y_pred.extend(tag_scores)
        
#         y_true.extend(prepare_sequence(tags, tag_to_ix).tolist())
#         tag_scores = model(prepare_sequence(sentence, word_to_ix))
#         for score in tag_scores:
#             val = max(range(len(score)), key = lambda x: score[x])
#             y_pred.append(val)
    val_micro_f1_model4.append(f1_score(y_true, y_pred, average='micro'))
    val_macro_f1_model4.append(f1_score(y_true, y_pred, average='macro'))
    val_weighted_f1_model4.append(f1_score(y_true, y_pred, average='weighted'))
    val_balanced_accuracy_model4.append(balanced_accuracy_score(y_true, y_pred))
    if (epoch == NUM_EPOCH-1):
        print ("Classification Report Validation")
        print(classification_report(y_true, y_pred))